In [1]:
#!/bin/python3
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
import copy

In [ ]:
cols = ["datatype", "targettype", "runID", "energy"]

    
    df = pd.read_csv(filename,sep='\t',names=["datatype","targettype","RunID","energy","d_reacted_carbon","all_carbon"])
